In [110]:
import os
import fitz
from tqdm import tqdm

In [111]:
def check_rate_function(rate_1, rate_2, possible_rate):
        # 檢查兩種方法提取的評價是否相同
    if rate_1 == rate_2 and rate_1!= None:
        return rate_1
    elif rate_1 in possible_rate:
        return rate_1
    elif rate_2 in possible_rate:
        return rate_2
    else :
        return 'NULL'
            

In [112]:
zip_path = '康和'
count_true = 0
possible_rate = ['買進 (維持評等)', '中性', '中性 (維持評等)', '買進(維持評等)', '中 性 (維 持 評 等 )', '中性 (調降評等)',
                 '買進 (調升評等)', '買進(首次評等)', '未評等', '未 評 等', '買進 (首次評等)', '新股掛牌 (未評等)', '新 股 掛 牌',
                 '買進-維持', '逢低買進-首次', '逢低買進', '逢低買進-維持', '買進-首次', '買進']
for directory in tqdm(os.listdir(zip_path)):
    directory_path = os.path.join(zip_path, directory)
    rate_1 = None
    rate_2 = None
    rate = None
    try:
        with fitz.open(directory_path) as doc:
            mat = fitz.Matrix(1, 1)
            page = doc.load_page(0)
            rect = page.rect
            page_check_source = doc.load_page(-1)
            text_check_source = page_check_source.get_text()
            if '康 和 投 資 顧 問' in text_check_source :
                # 檢查版本                
                clip_check_report = fitz.Rect(rect.width/2, 0, rect.width, 150)
                text_check_report = page.get_text(clip=clip_check_report)
                if any(keyword in text_check_report for keyword in ['個股報告', '個 股 報 告', '投  資  速  報', '投 資 速 報']):
                    # 提取評價的第一種方法
                    try:
                        clip_old_report_1 = fitz.Rect(0, 100, 400, 160)
                        text_old_report_1 = page.get_text(clip=clip_old_report_1, sort=True).strip()
                        pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_old_report_1)
                        pix.save("page-clip_new_report_1.png")
                        if '，' in text_old_report_1:
                            rate_1 = text_old_report_1.split('，')[0].strip()
                        elif '目標價' in text_old_report_1:
                            rate_1 = text_old_report_1.split('目標價')[0].strip()
                        else:
                            rate_1 = text_old_report_1.split('\n')[0].strip()
                        # rate_1 = text_old_report_1.split('\n')[0].strip()
                    except:
                        rate_1 = None
                    # 提取評價的第二種方法
                    # try:
                    #     clip_old_report_2 = fitz.Rect(230, 120, 430, 160)
                    #     rate_2 = page.get_text(clip=clip_old_report_2).strip()
                    # except:
                    #     rate_2 = None
                # else :
                #     print(f'{text_check_report}, {directory_path} NULL')
                rate = check_rate_function(rate_1, rate_2, possible_rate)
                if any(keyword in text_check_report for keyword in ['產業速報', '產 業 速 報']):
                    rate = 'wrong'
                
            else:
                # False
                print(f'{directory_path} is not posted by 康和投顧')
    except:
        print(directory_path, 'fail')
    if rate == 'NULL' :
        print('case1', directory_path, rate,
                f'rate_1={rate_1}, rate_2={rate_2}')
    elif rate == 'wrong':
        count_true += 1
        # print(directory_path, rate,
        #       f'text_check_new_report={text_check_new_report}, text_check_old_fast_report={text_check_old_fast_report}')
    elif rate != None :
        count_true += 1
        # print(directory_path, rate,
        #         f'rate_1={rate_1}, rate_2={rate_2}')
    else :
        print('case2', directory_path, rate,
                f'rate_1={rate_1}, rate_2={rate_2}')
# 計算成功和失敗的數量並打印
count_false = len(os.listdir(zip_path)) - count_true
success_rate = round((count_true/len(os.listdir(zip_path))), 3)*100
print(f'共{len(os.listdir(zip_path))}件, 成功{count_true}件, 失敗{count_false}件, 成功率:{success_rate}%')

100%|██████████| 110/110 [00:04<00:00, 22.62it/s]

共110件, 成功110件, 失敗0件, 成功率:100.0%


In [113]:
import fitz
doc = fitz.open(r'康和\2020_美亞_2019-04-22_康和_康和投顧-產業速報-自行車：ebike 2019~2020年續成長，維持長線看好巨大、美利達-20190422.pdf')
mat = fitz.Matrix(1, 1)
page = doc.load_page(0)  # loads page number 'pno' of the document (0-based)
rect = page.rect
pix = page.get_pixmap(matrix=mat, alpha=False)
pix.save("page-%i.png" % page.number)

In [114]:
# 檢查是否由第一金投顧出版
page_check_source = doc.load_page(-1)
text_check_source = page_check_source.get_text()
if '康 和 投 資 顧 問' in text_check_source :
    print('True')

True


In [115]:
# 檢查版本
clip_check_report = fitz.Rect(rect.width/2, 0, rect.width, 150)
pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_check_report)
pix.save("page-clip_check_report.png")
text_check_report = page.get_text(clip=clip_check_report)
if '個 股 報 告' in text_check_report :
    print('個 股 報 告')
else :
    print('NULL')
text_check_report

NULL


'長線看好巨\n台灣股市︱自行車 \n產 業 速 報 \n2019-04-22 \n'

In [116]:
# 報告為新版個股報告 提取評價的第一種方法
try:
    clip_old_report_1 = fitz.Rect(0, 100, 400, 160)
    text_old_report_1 = page.get_text(clip=clip_old_report_1, sort=True).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_old_report_1)
    pix.save("page-clip_new_report_1.png")
    rate_1 = text_old_report_1.split('，')[0].strip()
    # rate_1 = text_old_report_1.split('\n')[0].strip()
except:
    rate_1 = None
text_old_report_1

'大、美利達'

In [117]:
# 報告為新版個股報告 提取評價的第二種方法
try:
    clip_old_report_2 = fitz.Rect(0, 110, 400, 150)
    rate_2 = page.get_text(clip=clip_old_report_2).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_old_report_2)
    pix.save("page-clip_new_report_2.png")
except:
    rate_2 = None
rate_2

'大、美利達'